In [58]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime

In [2]:
tests = Tests('test1')
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Время последнего моделирования: 26/03/2021 20:02
Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [3]:
p0 = [0., 25., 90., 7.]
d_arr = [10., 20., 30., 40.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
alg_arr = ['ММП v2', 'Метод N пеленгов']
def target_func(seed=None):
            rng = np.random.RandomState(seed)
            b = 0
            d = rng.uniform(5, 50)
            c = rng.uniform(0, 180)
            v = rng.uniform(3, 15)
            return [b, d, c, v]
result_df = pd.DataFrame(columns=['Algorithm', 'std', 'sr_0', 'sr_1', 'sr_2', 'sr_3', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
mean_columns = ['Ка', 'Nf', 'Iter', 't']
max_columns = ['Nf', 'Iter', 't']

In [4]:
%%time
tma = TMA(observer)
for std in std_arr:
    tma.standart_deviation = np.radians(std)
    for alg in alg_arr:
        dict_results = tma.swarm(algorithm_name=alg, n=10000, target_func=target_func, p0=p0, seeded=True)
        df = tests.get_df(dict_results)
        tests.save_df(df, name='../tests/test2/' + str(alg) + ' ' + str(std) + '.xlsx')
        res = pd.DataFrame({'Algorithm': [alg], 'std': [std]})
        for i in range(4):
            res['sr_' + str(i)] = [df['Успех'].apply(lambda x: x[i]).mean(axis=0)]
        res[mean_columns] = df[mean_columns].mean(axis=0)
        res[['Nf_max', 'Iter_max', 't_max']] = df[max_columns].apply(max, axis=0)
        result_df = result_df.append(res)

..\project\botma.py:175: RuntimeWarning: invalid value encountered in sqrt
  perr = np.sqrt(np.diag(res[1]))
Wall time: 29min 12s


In [5]:
result_df.reset_index(drop=True)
Path("../tests/test2").mkdir(parents=True, exist_ok=True)
tests.save_df(result_df, name='../tests/test2/results2.xlsx')

In [6]:
display_df = result_df.copy()
precisions = {'Ка' : '{0:.2f}', 'Nf' : '{0:.0f}', 'Nf_max': '{0:.0f}',\
'Iter' : '{0:.0f}', 'Iter_max' : '{0:.0f}', 't' : '{0:.4f}', 't_max' : '{0:.4f}'}
for col, precision in precisions.items():
    display_df[col] = display_df[col].apply(lambda x: precision.format(x))

In [8]:
display_df

,Algorithm,std,sr_0,sr_1,sr_2,sr_3,Ка,Nf,Nf_max,Iter,Iter_max,t,t_max
0,ММП v2,0.0,0.9999,0.9999,0.9999,0.9999,0.08,10,51,9,36,0.0028,0.0436
0,Метод N пеленгов,0.0,1.0000,1.0000,1.0000,1.0000,0.00,nan,nan,nan,nan,0.0002,0.0083
0,ММП v2,0.1,0.9418,0.9828,0.9828,0.9924,0.09,10,54,9,38,0.0022,0.0153
0,Метод N пеленгов,0.1,0.5602,0.7582,0.7592,0.8478,0.01,nan,nan,nan,nan,0.0002,0.0099
0,ММП v2,0.2,0.8619,0.9417,0.9417,0.9677,0.12,10,60,9,42,0.0022,0.0331
0,Метод N пеленгов,0.2,0.3067,0.5167,0.5240,0.6141,0.05,nan,nan,nan,nan,0.0002,0.0082
0,ММП v2,0.3,0.7819,0.8998,0.8999,0.9363,0.17,10,57,9,40,0.0022,0.0102
0,Метод N пеленгов,0.3,0.1887,0.3514,0.3579,0.4422,0.11,nan,nan,nan,nan,0.0002,0.0086
0,ММП v2,0.5,0.6355,0.8220,0.8230,0.8770,0.33,10,55,9,39,0.0023,0.0273
0,Метод N пеленгов,0.5,0.0485,0.1462,0.1481,0.2284,0.38,nan,nan,nan,nan,0.0002,0.0085
